Introduction:  

The process of creating biodiesel is divided into multiple sections. Firstly, there is an extraction phase where different oil is extracted from different types of seeds with the following data in table below:

In [49]:
using DataFrames

# Creating an empty DataFrame with quoted column names
df1 = DataFrame(
    "Crop" => String[],
    "Yield [t/ha]" => Float64[],
    "Water demand [Ml/ha]" => Float64[],
    "Oil content [l/kg]" => Float64[]
)

# Adding rows to the DataFrame
push!(df1, ["Soybeans", 2.6, 5.0, 0.178])
push!(df1, ["Sunflower seeds", 1.4, 4.2, 0.216])
push!(df1, ["Cotton seeds", 0.9, 1.0, 0.433])

# Display the DataFrame
df1


Row,Crop,Yield [t/ha],Water demand [Ml/ha],Oil content [l/kg]
,String,Float64,Float64,Float64
1,Soybeans,2.6,5.0,0.178
2,Sunflower seeds,1.4,4.2,0.216
3,Cotton seeds,0.9,1.0,0.433


Thereafter, the vegetable oil is transferred to a transesterification process, where 1 liter of vegetable oil and 0.2 liters of methanol creates 0.9 liters of biodiesel. The price for price of methanol is €1.5/ liter.

Finally, biodiesel is mixed with petrol diesel to create the following products with the data according to table below :


In [50]:
using DataFrames

# Creating an empty DataFrame with quoted column names
df2 = DataFrame(
    "Product" => String[],
    "Bio diesel [%]" => Int[],
    "Price [€/l]" => Float64[],
    "Tax [%]" => Int[]
)

# Adding rows to the DataFrame
push!(df2, ["B5", 5, 1.43, 20])
push!(df2, ["B30", 30, 1.29, 5])
push!(df2, ["B100", 100, 1.16, 0])

# Display the DataFrame
df2

Row,Product,Bio diesel [%],Price [€/l],Tax [%]
,String,Int64,Float64,Int64
1,B5,5,1.43,20
2,B30,30,1.29,5
3,B100,100,1.16,0


The purchase price of the petrol diesel is €1/ liter.

1. Model

Set:

$I$ = \{B5, B30, B100\}, which is the set of product types.

$J$ = \{Biodiesel, Price, Tax\}, which are the subcategories for biodiesel products.

$K$ = \{Soybeans, Sunflower seeds, Cotton seeds\}, which is the set of  crop types.

$L$ = \{Yield, Water demand, Oil content\}, which is the set of subcategories for each crop type.

In [51]:
#Sets
Prod_I = 1:3 #3 types of refined biodiesels
Prod_cat_J = 1:3 #3 types of categories for each refined biodiesels NOT NEEDED?
Crop_K = 1:3     #3 types of crop types 
Crop_cat_L = 1:3 #3 types of categories for each crop type        NOT NEEDED?

#Labels
Prod = ["B5" "B30" "B100"]
Prod_cat =["Biodiesel" "Price" "Tax"]
Crop = ["Soybeans" "Sunflower seeds" "Cotton seeds"]
Crop_cat = ["Yield" "Water demand" "Oil content"]


#Data
Methanol_volume = 2/9 # Volume of methanol used per liter of biodiesel, which is 0.2/0.9 [l]
Methanol_price = 1.5 # Price of methanol per liter [€/l]
Petrol_price = 1 #[€/l]

#Crop data table
#Crop           Yield [t/ha]      Water demand [Ml/ha]       Oil content [l/kg]
Crop_data = [       2.6                    5.0                       0.178     #Soybeans      
                    1.4                    4.2                       0.216     #Sunflower seeds
                    0.9                    1.0                       0.433]    #Cotton seeds


#Product data table
#Product           Biodiesel [proportion]      Price [€/l]                 Tax [proportion]
Prod_data = [               0.05                  1.43                           0.2     #B5      
                            0.30                  1.29                           0.05    #B30
                            1.00                  1.16                           0]      #B100

Veg_volume = Prod_data[:,1]/0.9 #Volume of vegoil used in fuel type i

3-element Vector{Float64}:
 0.05555555555555556
 0.3333333333333333
 1.1111111111111112

Decision variables:

$A_k$ denotes the area [ha] of crop grown for type $k \in K$ grown.

$C_{k,l}$ denotes the data of column $l \in L$ for crop type $k \in K$.

$P_{i,j}$ denotes the data of column $j \in J$ for fuel type $i \in I$.

$R_{i}$ is the volume of fuel [liter] created for type $i \in I$

$B_i$ is the volume of petrol diesel [liter] used to make biodiesel for product type $i \in I$. The price of petrol diesel is €1/ liter.

$M$ is the total volume of methanol [liter] used. The price of methanol is €1.5/ liter.

$V_k$ is the volume of vegetable oil [liter] obtained from crop type for type $k \in K$.

$G_i$ is the volume of vegetable oil [liter] used to create 1 liter of product type $i \in I$.

Constraints:

Inequality constraints:

The inequality constraints make sure that we do not over exceed any physical limitations of the resources available. Therefore, the following constraints apply
$$\sum_{i \in I}B_i\leq 150 000,$$

which is the limit of petrol resources.
$$\sum_{k \in K} A_kC_{k,2} \leq 5000,$$

which limits the water resources to 5000 Ml.
$$\sum_{k \in K} A_{k} \leq 1600,$$

which is the available area limited to 1600 ha.
$$\sum_{i \in I} R_{i} \geq 280 000,$$

which satisfies the demand.

Equality (Flow) constraints:

In order for the production to create the correct amounts of product between the steps, the following flow constraints are needed within the steps.

Firstly, the amount of vegetable oil extracted from the seeds has to equal the seed content, which is 
$$V_k = 1000 A_kC_{k,1}C_{k,3} \text{, }\forall\, k \in K.$$

Another constraint is that the petrol diesel used in the process has to be the same as what is in the final product, denoted by
$$B_i = R_i(1-P_{i,1})\text{, }\forall\, i\in I.$$

From the transesterification process, we get the following equality constraint
$$M = \frac{2}{10}\sum_{k \in K}V_k,$$

which explains the proportion of vegetable oil and methanol.
Then there is the constraint of having equal amounts of vegetable oil in the refined biodiesel and the finalized products.
$$\sum_{i \in I} R_iP_{i,1}G_{i} = \sum_{k \in K} V_k.$$

Finally there is the non-negativity constraint
$$A_k, R_i, B_i, C_{k,l}, G_i, V_k \geq 0 \text{, } \forall k \in K, \forall i \in I \text{ and } l \in L.$$

Objective function:

We want to maximize the profit, which is formulated as
$$\text{max } z = \sum_{i \in I} (R_iP_{i,2}(1-P_{i,3}) - 1B_i)-1.5M$$

The equation takes into consideration the cost of the methanol and the petrol diesel in the first two terms. The summation is the profit made from selling the products while deducting the tax on each product.

In [52]:

function Bioprofit()
    include("Biodiesel_dat.jl")
    #Create an empty model
    m = Model()

    #Decision variables
    @variable(m, A_k[Crop_K]>=0) #Area [ha] of crop type k
    @variable(m, R_i[Prod_I]>=0) #Volume [liter] of final fuel of type i

    #B = sum(R_i[Prod_I]*(1-Prod_data[i,1]) for i in Prod_I) #Total volume [l] of petrol diesel used

    B_i = R_i.*(1 .- Prod_data[:,1])  #Volume [l] of petrol diesel used for fuel type i
    V_k = 1000 .*A_k.* Crop_data[:,1].* Crop_data[:,3]#Volume [l] of vegetable oil from crop type k
    M = sum(V_k[k].* Methanol_volume for k in Crop_K) #Volume of methanol used [l]
    Tax_deduct = (1 .- Prod_data[:,3])

    #Objective function
    @objective(m, Max, sum(R_i[i]* Prod_data[i,2]* Tax_deduct[i] - B_i[i]* Petrol_price for i in Prod_I)-M*Methanol_price)


    #Constraints
    #Inequality constraints
    @constraint(m, sum(B_i[i] for i in Prod_I) <= 150000) #Supply of petrol diesel limited
    @constraint(m, sum(A_k[k]*Crop_data[k,2] for k in Crop_K) <= 5000) #Water limit
    @constraint(m, sum(A_k[k] for k in Crop_K) <= 1600)   #Area limited
    @constraint(m, sum(R_i[i] for i in Prod_I) >= 280000)   #Demand constraint

    #Equality (flow) constraints
    @constraint(m, sum(R_i[i]* Veg_volume[i] for i in Prod_I) <= sum(V_k[k] for k in Crop_K))    

    return m, A_k, R_i, B_i, V_k, M
end


Bioprofit (generic function with 1 method)

2. Unedited results from the model:

After implementing the model constructed, the following values are obtained

    - the maximized profit is 548 163€,
    - the volume of fuel supplied in the order B5, B30 and B100 is [0, 214286, 552804] liters,
    - the volume of petrol diesel used is 150000 liters, where everything went into the production of B30,
    - the volume of methanol used is 137131 liters, 
    - the area of each crop type in the order soybeans, sunflower seeds, and cotton seeds is [850, 0, 750] ha,
    - the water usage of each crop type in the same order is [4250, 0, 750] Ml

Something to note is that all of the water, petrol diesel and area is used, which is a sign of the profit being maximized.

3. Sensitivity analysis

a. Resource availability

The goal is to figure out if the model becomes infeasible due to changes in the resource availability and if the area, water and petrol diesel availability are related. The starting profit, as described earlier, is  548 163€.

Firstly, the petrol diesel is analyzed. The starting value is 150 000 liters. This value can be decreased all the way to 0 and yet the model remains feasible. This means that only B100 is sold.

Secondly, the water limit is set to its maximum value of 5000 Ml. This value can be decreased to 371 Ml while the model still remains feasible.

Thirdly, the starting value for the area is set to 1600 ha. The area can be decreased to 313 ha while the model still remains feasible.

While performing this analysis, it is important to note that whenever the amount of petrol diesel decreased, the volume of  B$_{100}$ increased but area and water usage were not affected. This means that the model is not forced to include petrol and profit can be made of only B100. When decreasing the available area, water usage also decreases and only soybeans are grown. This causes the volume of B100 to decrease while leaving the volume of  B30 untouched. Due to the reduced production of products, the profit is decreased. Finally, when decreasing the water limit, the area used is also decreased. However, in this case, only cotton seeds are grown. This makes sense since it is the crop type that has the lowest water demand. Yet again, the volume of B30 remains untouched while B100 is reduced.

It seems that there is no profitability of producing B5, despite the changes in area and water limitations. This is due to the tax on the product being high.


b.  Marginal increments

In the following scenarios, the profit is analyzed by marginally increasing the supply of petrol diesel, water resources, and available area. These supplies are increased with a factor of 1\%, one at a time. When increasing the supply of petrol diesel, the following results are obtained

In [55]:
using DataFrames

# Creating an empty DataFrame with quoted column names
df3 = DataFrame(
    "Supply [l]" => Int[],
    "Profit [€]" => Int[]
)

# Adding rows to the DataFrame
push!(df3, [150000, 548163])
push!(df3, [151500, 551966])
push!(df3, [153019, 556150])
push!(df3, [154545, 560752])

# Display the DataFrame
df3

Row,Supply [l],Profit [€]
,Int64,Int64
1,150000,548163
2,151500,551966
3,153019,556150
4,154545,560752


From the marginal increments of the available petrol diesel, more fuel of type B30 is produced and sold, which increases the profit made. Therefore, by only increasing the available amount of petrol diesel, the profit will increase. Something to note is that no volume of B5 is still produced since it is still not profitable enough. Secondly, the area is marginally increased, which results in the following values

In [56]:
using DataFrames

# Creating an empty DataFrame with quoted column names
df4 = DataFrame(
    "Area [ha]" => Int[],
    "Profit [€]" => Int[]
)

# Adding rows to the DataFrame
push!(df4, [1600, 548163])
push!(df4, [1616, 552584])
push!(df4, [1632, 557005])
push!(df4, [1648, 561427])

# Display the DataFrame
df4

Row,Area [ha],Profit [€]
,Int64,Int64
1,1600,548163
2,1616,552584
3,1632,557005
4,1648,561427


There is a clear pattern that the more area is available, the more vegetable oil is produced, and therefore the supply of biodiesel increases. However, the proportion between B30 and B100 changes since no more petrol diesel is available, the only product that can be produced is B100. Therefore, more B100 is produced and increases the profit made. Finally, the water supply is also marginally increased, which ends up in the values according to the table below

In [57]:
using DataFrames

# Creating an empty DataFrame with quoted column names
df5 = DataFrame(
    "Water supply [Ml]" => Int[],
    "Profit [€]" => Int[]
)

# Adding rows to the DataFrame
push!(df5, [5000, 548163])
push!(df5, [5050, 548842])
push!(df5, [5100, 549522])
push!(df5, [5150, 550202])

# Display the DataFrame
df5

Row,Water supply [Ml],Profit [€]
,Int64,Int64
1,5000,548163
2,5050,548842
3,5100,549522
4,5150,550202


According to the values, an increase in water supply also increases the profit, which is reasonable. This is due to more crops planted, which increases the volume of biodiesel produced and therefore the volume of B100 sold. No more B30 can be produced since all of the petrol diesel supply is used for the production of B30. Yet again, no B5 is produced due to its low profitability. Something interesting to note is that the amount of soybeans planted will always increase as the water supply increases. It reaches a point where cotton seeds are not as profitable and fewer are planted. In general, soybeans offer the greatest yield but are restricted by their highest water demand. By being less strict with the water constraint, it seems obvious to plant as much of it as possible and offer less area for cotton seeds. 

c. Non-zero sunflower planting

Chose to only analyze the sunflower seeds since the problem lies with it. The accuracy was also limited to the same number of figures of value as was given for each of the parameters. 

Removing the need for water completely still gives a zero solution indicating that it does not have a major effect on this problem. If we instead adjust the yield we get a non-zero solution once the value passes 2.08, and the area for the crops [Soybeans, Sunflower seeds, Cotton seeds] is [0, 1062.5, 537.5], in other words, the soybeans will get a zero-solution instead.
While adjusting the oil content the first non-zero solution for the sunflower seeds once again put the soybeans area to zero. The area used by the crops is [0 1062.6 537.5], exactly the same as when adjusting the yield indicating that there is a relation between the area used by the soybeans and sunflower seeds.

d.  Increased petrol diesel cost
Adjusting the petrol diesel cost to $1.2 €$ gives the product type as [B5, B30, B100] = [0,  214285.514,  552803.787], an area distribution of [850,  0,  750] for the crop types [Soybeans, Sunflower seeds, Cotton seeds], resulting in a profit of 518163.03 €


e. Tax changes

Adjusting the $20\,\%$ tax on B5 gives significant changes to the objective value as the tax approaches zero.

In [58]:
using DataFrames

# Creating an empty DataFrame with quoted column names
df6 = DataFrame(
    "Biodiesel type" => String[],
    "Tax [%]" => Int[],
    "Price [€/l]" => Float64[]
)

# Adding rows to the DataFrame
push!(df6, ["B5", 50, 548163])
push!(df6, ["B5", 80, 548163])
push!(df6, ["B5", 30, 548163])
push!(df6, ["B5", 5, 565469])
push!(df6, ["B5", 0, 576759])

# Display the DataFrame
df6

Row,Biodiesel type,Tax [%],Price [€/l]
,String,Int64,Float64
1,B5,50,548163.0
2,B5,80,548163.0
3,B5,30,548163.0
4,B5,5,565469.0
5,B5,0,576759.0


Adjusting the $5\,\%$ tax on B30  shows that it does not really have a major effect on the objective value since it barely changes even with drastic alterations to the tax. 

In [59]:
using DataFrames

# Creating an empty DataFrame with quoted column names
df7 = DataFrame(
    "Biodiesel type" => String[],
    "Tax [%]" => Int[],
    "Price [€/l]" => Float64[]
)

# Adding rows to the DataFrame
push!(df7, ["B30", 50, 531601])
push!(df7, ["B30", 1, 559220])
push!(df7, ["B30", 80, 531601])
push!(df7, ["B30", 0, 561984])
push!(df7, ["B30", 30, 531601])

# Display the DataFrame
df7

Row,Biodiesel type,Tax [%],Price [€/l]
,String,Int64,Float64
1,B30,50,531601.0
2,B30,1,559220.0
3,B30,80,531601.0
4,B30,0,561984.0
5,B30,30,531601.0


Adjusting the $0\,\%$ tax on B100 leads to a major change to the objective value after changing the tax only a few percent, more exactly, around $7\,\%$

In [60]:
using DataFrames

# Creating an empty DataFrame with quoted column names
df8 = DataFrame(
    "Biodiesel type" => String[],
    "Tax [%]" => Int[],
    "Price [€/l]" => Float64[]
)

# Adding rows to the DataFrame
push!(df8, ["B100", 5, 516100])
push!(df8, ["B100", 20, 419913])
push!(df8, ["B100", 50, 227537])
push!(df8, ["B100", 10, 484038])
push!(df8, ["B100", 7, 503275])
push!(df8, ["B100", 9, 490450])

# Display the DataFrame
df8

Row,Biodiesel type,Tax [%],Price [€/l]
,String,Int64,Float64
1,B100,5,516100.0
2,B100,20,419913.0
3,B100,50,227537.0
4,B100,10,484038.0
5,B100,7,503275.0
6,B100,9,490450.0


f. Varied water demand

To alter the need for water equally for each plant at the same time, the water demand was multiplied with a constant factor that was varied in a range between $0.5$ and $1.5$.

In [61]:
using DataFrames

# Creating an empty DataFrame with quoted column names
df9 = DataFrame(
    "Scale factor" => Float64[],
    "Soybeans [ha]" => Float64[],
    "Sunflower seeds [ha]" => Int[],
    "Cotton seeds [ha]" => Float64[],
    "Price [€/l]" => Float64[]
)

# Adding rows to the DataFrame
push!(df9, [1.0, 850, 0, 750, 548163])
push!(df9, [1.5, 433.33, 0, 1166.67, 119064])
push!(df9, [0.5, 1600, 0, 0, 526089])
push!(df9, [1.1, 736.36, 0, 863.63, 485051])
push!(df9, [1.2, 641.66, 0, 958.33, 480551])
push!(df9, [1.3, 561.5, 0, 1038.5, 476743])

# Display the DataFrame
df9

Row,Scale factor,Soybeans [ha],Sunflower seeds [ha],Cotton seeds [ha],Price [€/l]
,Float64,Float64,Int64,Float64,Float64
1,1.0,850.0,0,750.0,548163.0
2,1.5,433.33,0,1166.67,119064.0
3,0.5,1600.0,0,0.0,526089.0
4,1.1,736.36,0,863.63,485051.0
5,1.2,641.66,0,958.33,480551.0
6,1.3,561.5,0,1038.5,476743.0



The conclusion that can be drawn from these alterations is that a change of about 10-20\,\% is needed to have larger changes for the soybean plantation.

g. Altered formulas

In this section, it is investigated how the optimal solution changes if the data of the problem is altered.

Assume that the right-hand-sides of the constraints make a small increment/ reduction of one of the limiting resources. The optimal solution will change and therefore the profit is also affected. Hence, the shadow price of that value is this amount. The optimal solution will not change since an increase larger than a specified size in the value gives no further income.

One way to compute the shadow price is to construct a dual problem of our income model (which is the primal problem) and in the linear programming dual, observe that the dual problem is in accordance with the Lagrangian duality theorem. 

The optimal dual solution is indeed the shadow price for the constraints. If a value on the right-hand side, $b_i$, changes, the optimal basis does not change. However, the optimal value will change exactly with the value $y^*_i$, where $y^*$ is the optimal solution of the dual problem. The formula for the reduced cost is $j^* = \text{arg min}_{j \subset N}{c_j - c_B^TB^{-1}}A_J$


Furthermore, if the necessary condition for the basis remains optimal, then $B^{-1}(b +\Delta b)$ is non-negative, which means the values remain within the feasible set of points.

h. A more sustainable objective function

There are multiple benefits of having sustainable biodiesel production. Some examples are

    - a reduction in life-cycle greenhouse gases,
    - reduction in hydrocarbon emissions.

 Therefore, conservation-oriented production methods (with no tillage, less water, and fertilizer consumption) with good crop selection are crucial for producing better biofuels. One reason is that using less water and more efficient ways to apply fertilizer together with no tillage and an efficient harvest reduces the net carbon emissions. Efficient ways of applying fertilizer are crucial because it reduces the outgassing of greenhouse gases that occur after fertilization. Therefore, it is necessary to take these methods into account in order to maximize the profit while paying attention to the emission of greenhouse gases.

Another way of improving the model is to add a constraint limiting the volume of greenhouse gases that are released. To do this, data containing the volume of gases released during the process of producing biodiesel. To further build upon this thought, adding a penalty fee for the types of diesel containing more petrol would improve the model even further, but with the risk of a lowered profit. The objective function will therefore be on the following form

$$\text{max } z_{new} = \sum_{i \in I} (R_iP_{i,2}(1-P_{i,3}) - 1B_i)-1.5M - Y(\sum_{i \in I}(B_i) + M),$$

where $Y$ is a penalty constant. 

The reasoning behind the new objective function is that petrol diesel and methanol contribute the most to greenhouse gas emissions. Therefore, by paying an extra fee that is proportional to the volume of gases emitted, the production can be penalized for the more gases it releases.

Alternatively, change the objective function to minimize the release of greenhouse gases and change the profit to only be a constraint.